<a href="https://colab.research.google.com/github/ayanand/CapstoneML/blob/main/analysiscap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ayanand/CapstoneML/blob/main/analysiscap.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [2]:
import pandas as pd
!pip install fsspec
!pip install gcsfs 
 
 


In [4]:
df = pd.read_csv('gs://capstoragebucket2020/All_joined_v4.csv')

In [5]:
df.tail()

,latitude,longitude,quarter,year,incendie_count,alarmes_incendies_count,total_crimes,vols_count,mefait_count,vol_de_vehicule_count,introduction_count,vol_moteur_count,infractions_entrainant_count,sum_etage_hors_sol,sum_nombre_logement,min_annee_construction,max_annee_construction,avg_annee_construction,sum_superficie_terrain,area,population,dwellings,households,averagehouseholdsize,averageage,averagesizeofcensusfamilies,workers,caserne_count
11838,45.7,-73.48,3,2019,0,0,6,0,0,2,2,2,0,480,421,1890,2018,1975.681388,173165,0.32098,657,271,262,2.3,46.1,2.8,160,0
11839,45.7,-73.48,4,2019,0,0,2,0,0,0,1,1,0,480,421,1890,2018,1975.681388,173165,0.32098,657,271,262,2.3,46.1,2.8,160,0
11840,45.7,-73.48,1,2020,0,2,0,0,0,0,0,0,0,480,421,1890,2018,1975.681388,173165,0.32098,657,271,262,2.3,46.1,2.8,160,0
11841,45.7,-73.48,2,2020,0,3,1,0,1,0,0,0,0,480,421,1890,2018,1975.681388,173165,0.32098,657,271,262,2.3,46.1,2.8,160,0
11842,45.7,-73.48,3,2020,0,1,6,0,1,2,0,3,0,480,421,1890,2018,1975.681388,173165,0.32098,657,271,262,2.3,46.1,2.8,160,0


In [6]:
df=df.sort_values(by=['year', 'quarter','latitude','longitude'])

In [7]:
print(df['incendie_count'].value_counts())


#Define three classes of fire risk by the summed number of fire incidence from 2015-2020
#low risk < 5, moderate risk 5-15, high risk > 15
df['fire'] = df['incendie_count']

df['fire'][df['incendie_count']==0] = 0
df['fire'][(df['incendie_count']>=1) & (df['incendie_count']<4)] = 1
df['fire'][df['incendie_count']>=4] = 2



  

0     7633
1     2644
2      961
3      362
4      151
5       55
6       19
7        9
9        4
8        4
10       1
Name: incendie_count, dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [8]:
df_new = df
df_new['year'] =2021- df_new['year']

In [9]:
import geopy.distance

def dist(x,y):
  coords_1 = (45.504654, -73.56546)
  coords_2 = (x, y)
  return geopy.distance.vincenty(coords_1, coords_2).km


In [10]:
df_new['distance']=0

In [11]:
for x in range(0,len(df_new)-1):
  df_new['distance'][x]=dist(df_new['latitude'][x], df_new['longitude'][x])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
df_new.tail(10)

,latitude,longitude,quarter,year,incendie_count,alarmes_incendies_count,total_crimes,vols_count,mefait_count,vol_de_vehicule_count,introduction_count,vol_moteur_count,infractions_entrainant_count,sum_etage_hors_sol,sum_nombre_logement,min_annee_construction,max_annee_construction,avg_annee_construction,sum_superficie_terrain,area,population,dwellings,households,averagehouseholdsize,averageage,averagesizeofcensusfamilies,workers,caserne_count,fire,distance
11545,45.66,-73.51,4,1,0,0,5,0,0,4,0,1,0,1207,1044,1941,2017,1984.169195,494180,0.28223,448,168,167,2.6,41.7,3.0,135,0,0,17
11569,45.66,-73.50,4,1,0,0,2,0,1,1,0,0,0,1953,1624,1925,2014,1976.802276,557374,0.30364,1090,395,388,2.8,39.9,3.0,290,0,0,18
11593,45.66,-73.49,4,1,0,0,1,0,0,1,0,0,0,490,542,1752,2018,1976.853933,178758,0.21275,487,212,210,2.2,47.9,2.6,110,0,0,18
11617,45.67,-73.55,4,1,0,0,1,0,0,1,0,0,0,874,607,1900,2017,1994.664671,196023,0.72304,453,158,154,2.7,43.1,2.7,75,0,0,18
11685,45.67,-73.51,4,1,0,2,2,0,0,0,0,2,0,459,576,1920,2015,1981.962264,338851,0.73267,720,345,343,2.0,47.3,2.7,165,1,0,18
11709,45.67,-73.50,4,1,0,0,2,0,0,0,1,1,0,1706,2237,1914,2019,1969.399127,518847,0.11503,505,285,273,1.8,43.2,2.6,100,0,0,19
11738,45.68,-73.50,4,1,0,0,5,0,1,3,0,1,0,1377,898,1959,2019,1994.756098,465540,0.45534,821,422,386,2.1,37.5,2.7,335,0,0,20
11760,45.69,-73.50,4,1,0,0,2,0,0,0,2,0,0,254,285,1959,2018,1999.281967,75473,2.99229,1350,519,518,2.6,38.5,2.9,580,0,0,21
11784,45.69,-73.49,4,1,0,0,3,0,0,1,0,2,0,1754,1687,1925,2017,1990.051636,493065,0.20290,677,290,286,2.4,40.4,2.7,195,0,0,21
11819,45.70,-73.49,4,1,0,0,1,0,0,0,0,1,0,933,805,1900,2019,1997.866756,177975,0.48567,645,255,249,2.6,41.3,2.9,170,1,0,22


In [13]:

df_new = df_new.drop("latitude", axis=1)
df_new = df_new.drop("longitude", axis=1)

In [14]:
df_new = df_new.drop("min_annee_construction", axis=1)
df_new = df_new.drop("max_annee_construction", axis=1)
df_new = df_new.drop("incendie_count", axis=1)

In [15]:
#y= df_new["incendie_count"].copy()
y= df_new["fire"].copy()
X=df_new.drop("fire", axis=1)

In [16]:
y.value_counts()

0    7633
1    3967
2     243
Name: fire, dtype: int64

In [17]:
len(X),len(y)

(11843, 11843)

In [18]:
corr_matrix = df_new.corr()
corr_matrix["fire"].sort_values(ascending=False)

fire                            1.000000
sum_nombre_logement             0.479032
total_crimes                    0.429216
sum_etage_hors_sol              0.420503
alarmes_incendies_count         0.410367
mefait_count                    0.399054
introduction_count              0.395315
vols_count                      0.312964
vol_moteur_count                0.309208
vol_de_vehicule_count           0.209143
caserne_count                   0.158662
sum_superficie_terrain          0.084048
dwellings                       0.061831
infractions_entrainant_count    0.052661
households                      0.036779
year                            0.014774
quarter                        -0.031297
averageage                     -0.038386
population                     -0.051500
averagesizeofcensusfamilies    -0.071044
workers                        -0.083336
averagehouseholdsize           -0.167736
area                           -0.202429
avg_annee_construction         -0.207849
distance        

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

scaler = StandardScaler()
scaler.fit(X)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [20]:
X=scaler.transform(X)

In [21]:
X

array([[-1.33768027,  1.45578401, -0.73183283, ..., -1.19958877,
        -0.38140348,  2.71451408],
       [-1.33768027,  1.45578401, -0.61567372, ...,  1.22125921,
        -0.38140348,  2.71451408],
       [-1.33768027,  1.45578401, -0.73183283, ..., -0.20471974,
        -0.38140348,  2.57144845],
       ...,
       [ 1.35992686, -1.49197451, -0.73183283, ...,  2.64723816,
        -0.38140348,  1.28385784],
       [ 1.35992686, -1.49197451, -0.73183283, ...,  0.09374097,
        -0.38140348,  1.28385784],
       [ 1.35992686, -1.49197451, -0.73183283, ..., -0.07207053,
         2.62189533,  1.42692346]])

In [22]:
X=pd.DataFrame(X)

In [23]:
X_train=X[df_new['year'] > 2] # choose 2019 data as train
# X_test =X[df_new['year'] == 2] # choose 2019 data as validation
y_train = y[df_new['year'] > 2]
# y_test = y[df_new['year'] == 2]

X_test = X[df_new['year']*df_new['quarter'] == 2]
y_test = y[df_new['year']*df_new['quarter'] == 2]

len(X_train),len(X_test),len(y_train),len(y_test)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(7999, 985, 7999, 985)

In [24]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

#X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1,random_state=4)

len(X_train),len(X_test),len(y_train),len(y_test)

(7999, 985, 7999, 985)

In [25]:
y_train.value_counts(), y_test.value_counts()

(0    5111
 1    2730
 2     158
 Name: fire, dtype: int64, 0    594
 1    359
 2     32
 Name: fire, dtype: int64)

In [26]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [27]:
lin_reg.coef_

array([-0.00987946, -0.00619627, -0.00058286,  0.00195958, -0.01725819,
        0.01468588,  0.00633329, -0.00913029,  0.0071238 ,  0.00530306,
        0.01103089, -0.02182501, -0.00092116,  0.0074872 , -0.00269852,
       -0.00320891, -0.06158341,  0.03598371, -0.01136817,  0.00201971,
        0.00922054,  0.03299613,  0.00827337, -0.01443288])

In [28]:
print(lin_reg.score(X_train, y_train))

0.0034505455696143272


In [29]:

print(lin_reg.score(X_test,y_test))

-0.013137920992844167


In [30]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42,max_depth=5)
tree_reg.fit(X_train, y_train)

DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=5,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=42, splitter='best')

In [31]:
from sklearn.metrics import mean_squared_error
import numpy as np

y_predictions = tree_reg.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.5604106921284754

In [32]:
Xyz=pd.DataFrame(y_predictions)

Xyz.value_counts(), y_test.value_counts()

(0.381738    684
 0.333333    153
 0.491736     59
 0.361039     58
 0.568421     10
 0.146667     10
 0.476190      3
 0.437500      3
 0.377778      2
 0.193548      2
 0.750000      1
 dtype: int64, 0    594
 1    359
 2     32
 Name: fire, dtype: int64)

In [33]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42, max_depth=10)
forest_reg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=10, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [34]:
print(str(forest_reg.score(X_train, y_train)) + " "+ str(forest_reg.score(X_test, y_test) ))

0.17484114049201815 -0.041123344432210995


In [35]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
# prepare the cross-validation procedure
cv = KFold(n_splits=5, random_state=1, shuffle=True)
# create model
xgboost = XGBClassifier()
# evaluate model
xgboost_scores = cross_val_score(xgboost, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (mean(xgboost_scores ), std(xgboost_scores )))

Accuracy: 0.634 (0.005)


In [36]:

xgboost.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [37]:
y_test_pred = xgboost.predict(X_test)

In [38]:

print('Accuracy of XGboost classifier on test set: {:.2f}'.format(xgboost.score(X_test, y_test)))

Accuracy of XGboost classifier on test set: 0.60


In [39]:
from sklearn.metrics import confusion_matrix
confusion_matrix_valid = confusion_matrix(y_test, y_test_pred)
print(confusion_matrix_valid)

[[588   6   0]
 [357   2   0]
 [ 32   0   0]]


In [40]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly_features.fit_transform(X_train)



In [41]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y_train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [42]:
print(lin_reg.score(X_poly, y_train))

0.27440677709213723


In [43]:
print(lin_reg.score(X_poly, y_train))

0.27440677709213723


In [44]:
from sklearn.svm import SVC


svm_clf=SVC()
svm_clf.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [45]:
y_predictions=svm_clf.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.7067477525870188
[[587   7   0]
 [357   2   0]
 [ 32   0   0]]


In [46]:
cross_val_score(svm_clf,X_test,y_test,cv=3,scoring="accuracy")

array([0.59878419, 0.60060976, 0.58841463])

In [47]:
from sklearn.svm import LinearSVC

svm_clf2 = LinearSVC(max_iter=10000)
svm_clf2.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=10000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [48]:
y_predictions=svm_clf2.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.7031473840204643
[[594   0   0]
 [359   0   0]
 [ 32   0   0]]


In [49]:
from sklearn.svm import SVC

svm_clf2 = SVC(kernel="poly", degree =3, coef0=1,C=5)
svm_clf2.fit(X_train,y_train)

SVC(C=5, break_ties=False, cache_size=200, class_weight=None, coef0=1,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [50]:
y_predictions=svm_clf2.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.7067477525870188
[[568  24   2]
 [334  24   1]
 [ 31   1   0]]


In [51]:
%tensorflow_version 2.x
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD

In [52]:
#K.clear_session()

model = Sequential()
model.add(Dense(1, input_shape=(24,), activation='softmax'))
model.add(Dense(10, activation='softmax'))
model.add(Dense(10, activation='softmax'))
model.add(Dense(10, activation='softmax'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

history = model.fit(X_train, y_train)
result = model.evaluate(X_test, y_test)

31/31 [==============================] - 0s 938us/step - loss: 0.7033 - accuracy: 0.6030


In [53]:
#test softmax regression

from sklearn.linear_model import LogisticRegression

softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10, random_state=42,max_iter=1000)
softmax_reg.fit(X_train, y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=42, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [54]:
y_predictions=softmax_reg.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.7031473840204643
[[594   0   0]
 [359   0   0]
 [ 32   0   0]]


In [55]:
## polynomial kernel SVM


from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import LinearSVC

polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)),
        ("svm_clf", LinearSVC(C=10, loss="hinge", random_state=42))
    ])

polynomial_svm_clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=42, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [56]:
y_predictions=polynomial_svm_clf.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.8066978500775844
[[324 247  23]
 [209 142   8]
 [ 18  13   1]]


In [57]:
## polynomial kernel ; kernel trick ??


from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
        ("svm_clf", SVC(kernel="poly", degree=5, coef0=10, C=5))
    ])
## poly_kernel_svm_clf.fit(X_train, y_train)## taking lot of time

In [61]:
# gaussian svm

rbf_kernel_svm_clf = Pipeline([
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
rbf_kernel_svm_clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('svm_clf',
                 SVC(C=0.001, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [63]:
y_predictions=rbf_kernel_svm_clf.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.7031473840204643
[[594   0   0]
 [359   0   0]
 [ 32   0   0]]


In [64]:
## svm regression
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X_train, y_train)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=42, tol=0.0001, verbose=0)

In [67]:
print(svm_reg.score(X_test,y_test))

-0.016059183402084676


In [77]:

from sklearn.tree import DecisionTreeClassifier



tree_clf = DecisionTreeClassifier(max_depth=100, random_state=42)
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=100, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [79]:
y_predictions=tree_clf.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.8198055978548279
[[351 218  25]
 [238 111  10]
 [ 22   8   2]]


In [83]:
## ensemble

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression(solver="lbfgs", random_state=42,max_iter=1000)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)],
    voting='hard')

In [84]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=42,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                              

In [85]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.6030456852791878
RandomForestClassifier 0.5604060913705584
SVC 0.5979695431472081
VotingClassifier 0.601015228426396


bagging ensemble

In [86]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(random_state=42), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [87]:
y_predictions=bag_clf.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.7024250958133731
[[594   0   0]
 [358   1   0]
 [ 32   0   0]]


In [88]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=16, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [89]:
y_predictions=rnd_clf.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.7031473840204643
[[594   0   0]
 [359   0   0]
 [ 32   0   0]]


In [90]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [91]:
y_predictions=ada_clf.predict(X_test)
tree_mse = mean_squared_error(y_test, y_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)
confusion_matrix_valid = confusion_matrix(y_test, y_predictions)
print(confusion_matrix_valid)

0.7067477525870188
[[584  10   0]
 [354   5   0]
 [ 32   0   0]]
